<hr style="height:2px;">

# Training data generation for denoising of Tribolium

We demonstrate training data generation for the case where both low SNR and corresponding high SNR stacks can be acquired.  


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
from tifffile import imread
from csbdeep import datagen
from csbdeep.plot_utils import plot_some

<hr style="height:2px;">

# Raw data

Corresponding low and high stacks should have identical names but located in different folders. 
We only show one stack for demonstration, but you should have ~20-50 different stacks with at different developmental timepoints.

In [ ]:
ls raw_data/tribolium/*

Lets just plot the low and high (GT) stacks:

In [ ]:
y = imread('raw_data/tribolium/GT/nGFP_0.1_0.2_0.5_20_13_late.tif')
x = imread('raw_data/tribolium/low/nGFP_0.1_0.2_0.5_20_13_late.tif')
print('image size =', y.shape)

plt.figure(figsize=(15,10))
plot_some(np.stack([x,y]),title_list=[['input','output']], pmin=2,pmax = 99.8);

<hr style="height:2px;">

# Patch generation

From corresponding stacks, we generate 3D patches of size (16,128,128) that are sampled from regions of intersting structures. 

In [ ]:
X,Y = datagen.create_patches(p='data/tribolium/', n_samples=128, patch_size=(16,128,128),thresh_patch=0.4, inputs=["low"])
print("shape of X", X.shape)
print("shape of Y", Y.shape)

This shows the maximum projection of some of those patch pairs (even rows are inpout, odd rows output)

In [ ]:
from csbdeep.plot_utils import plot_some

for i in range(3):
    plt.figure(figsize=(16,4))
    sl = slice(8*i, 8*(i+1))
    plot_some(X[sl],Y[sl], pmin=2, pmax=99.8)
    plt.show()
None;